## Example Usage of the GMail Python API

This Notebook shows the basic usage of the GMail API

1. Load E-Mails from the server

2. Send E-Mails with the server

In order to be able to use the API you need a valid token as a .pickle file. In the first initialization of the GMail service you need to pass the path to this token.pickle file. Change the parameter below to your file to rerun the examples. 


For furhter information on how to send attachments, ... please consult the Documentation: 

https://googleapis.github.io/google-api-python-client/docs/epy/index.html

and the Guide: 

https://developers.google.com/gmail/api/quickstart/python

In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
from googleapiclient.discovery import build, Resource
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import base64
import pickle
import os
from typing import Dict
import email
import pandas as pd

### Basic Functions

In [3]:
def load_service(token_path:str):
    """ Load the service from the specifies token file

    Args:
        token_path (str): Authorized Gmail API service instance.
    """
    if os.path.exists(token_path):
        with open(token_path, 'rb') as token:
            creds = pickle.load(token)
            service = build('gmail', 'v1', credentials=creds)
            return service

    else: 
        print("Credentials not found")
        return None

In [4]:
def send_message(service:Resource, user_id:str, message:Dict[str,str]):
    """Send an email message.

    Args:
        service (Resource): Authorized Gmail API service instance.
        user_id (str): User's email address. The special value "me" can be used to indicate the authenticated user.
        message (Dict[str,str]): Message to be sent.
    """
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                  .execute())
        print('Message Id: %s' % message['id'])
    except Exception as e:
        print('An error occurred: %s' % e)

In [5]:
def create_message(sender:str, to:str, subject:str, message_text:str) -> Dict[str,str]:
  """Create a message for an email.

  Args:
    sender (str): Email address of the sender.
    to (str): Email address of the receiver.
    subject (str): The subject of the email message.
    message_text (str): The text of the email message.

  Returns:
    Dict[str,str]: An object containing a base64url encoded email object.
  """
  message = MIMEMultipart() #MIMEText(message_text, "html", _charset="utf-8")
  #message.set_payload(message_text, charset="utf-8")
  message['to'] = to
  message['from'] = sender
  message['subject'] = subject
  message.attach(MIMEText(message_text, _charset='utf-8'))

  print(message)

  encoded_message = base64.urlsafe_b64encode(message.as_bytes())
  print(encoded_message)
  print(encoded_message.decode())
  return {'raw': encoded_message.decode()}


In [6]:
message = create_message("datathon@lumos-consulting.at", "thomas.n71@gmail.com", "Test_0", "Das ist der erste Testä")

Content-Type: multipart/mixed; boundary="===============8990634996275285065=="
MIME-Version: 1.0
to: thomas.n71@gmail.com
from: datathon@lumos-consulting.at
subject: Test_0

--===============8990634996275285065==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64

RGFzIGlzdCBkZXIgZXJzdGUgVGVzdMOk

--===============8990634996275285065==--

b'Q29udGVudC1UeXBlOiBtdWx0aXBhcnQvbWl4ZWQ7IGJvdW5kYXJ5PSI9PT09PT09PT09PT09PT04OTkwNjM0OTk2Mjc1Mjg1MDY1PT0iCk1JTUUtVmVyc2lvbjogMS4wCnRvOiB0aG9tYXMubjcxQGdtYWlsLmNvbQpmcm9tOiBkYXRhdGhvbkBsdW1vcy1jb25zdWx0aW5nLmF0CnN1YmplY3Q6IFRlc3RfMAoKLS09PT09PT09PT09PT09PT04OTkwNjM0OTk2Mjc1Mjg1MDY1PT0KQ29udGVudC1UeXBlOiB0ZXh0L3BsYWluOyBjaGFyc2V0PSJ1dGYtOCIKTUlNRS1WZXJzaW9uOiAxLjAKQ29udGVudC1UcmFuc2Zlci1FbmNvZGluZzogYmFzZTY0CgpSR0Z6SUdsemRDQmtaWElnWlhKemRHVWdWR1Z6ZE1PawoKLS09PT09PT09PT09PT09PT04OTkwNjM0OTk2Mjc1Mjg1MDY1PT0tLQo='
Q29udGVudC1UeXBlOiBtdWx0aXBhcnQvbWl4ZWQ7IGJvdW5kYXJ5PSI9PT09PT09PT09PT09PT04OTkwNjM0OTk2Mjc1Mjg1MDY1

In [7]:
def receive_unread_messages(service:Resource):
    """ Recieve all unread messages and print them

        Args:
        service (Resource): Authorized Gmail API service instance.

    """
    messages = []
    # request a list of all the messages 
    result = service.users().messages().list(userId='me').execute() 
  
    # iterate through all the messages 
    for msg in result.get('messages') : 
        # Get the message from its id   
        # Use try-except to avoid any Errors 
        message = service.users().messages().get(userId="me", id=msg["id"],
                                             format='raw').execute()
        
        
        if "UNREAD" in message["labelIds"]:
            try:
                print(message["id"])
                msg_str = base64.urlsafe_b64decode(message['raw'].encode("utf-8")).decode("utf-8")
                print(msg_str)
                mime_msg = email.message_from_string(msg_str)
                
                messages.append(mime_msg)
    
            except: 
                print("error")
                pass
    return messages

In [8]:
def receive_all_messages(service:Resource):
    """ Recieve all messages and print them

        Args:
        service (Resource): Authorized Gmail API service instance.

    """
    messages = []
    # request a list of all the messages 
    result = service.users().messages().list(userId='me').execute() 
  
    # iterate through all the messages 
    for msg in result.get('messages') : 
        # Get the message from its id   
        # Use try-except to avoid any Errors 
        message = service.users().messages().get(userId="me", id=msg["id"],
                                             format='raw').execute()
        
        
        
        try:
            print(message["id"])
            msg_str = base64.urlsafe_b64decode(message['raw'].encode("utf-8")).decode("utf-8")
            print(msg_str)
            mime_msg = email.message_from_string(msg_str)

            messages.append(mime_msg)

        except: 
            print("error")
            pass
    return messages

In [9]:
def print_message(message):
    """ Print the Content of a Message

    Args:
        message (email): The email encoded message
    """

    if message.is_multipart():
        for payload in message.get_payload():
            print(payload.get_payload())
    else:
        print(message.get_payload())

### Eigene Funktionen

In [10]:
def markAsRead(service:Resource, message_id):
    if type(message_id) is list:
        for mid in message_id:
            service.users().messages().modify(userId='me', id=mid, body={'removeLabelIds': ['UNREAD']}).execute()
    else:
        service.users().messages().modify(userId='me', id=message_id, body={'removeLabelIds': ['UNREAD']}).execute()
    
def deleteMail(service:Resource, message_id):
    if type(message_id) is list:
        for mid in message_id:
            service.users().messages().delete(userId="me", id=mid).execute()
    else:
        service.users().messages().delete(userId="me", id=message_id).execute()

In [11]:
# Nötig, wenn Nachrichten von Google reinkommen.
# deleteMail(service,"17767b037c45a4d3")

### Usage of the function

In [12]:
token_path = "token.pickle"

In [13]:
service = load_service(token_path)
messages = receive_all_messages(service)

17773d095a0a02e1
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 12:09:45 -0800
Content-Type: multipart/mixed; boundary="===============6504758777782277436=="
MIME-Version: 1.0
to: datathon@lumos-consulting.at
From: team4@lumos-consulting.at
subject: Test_0
Date: Fri, 5 Feb 2021 12:09:45 -0800
Message-Id: <CAEzfA9rFeu13z=4Ffkf3UqH5H15eyfFYk5vv0ov-dEO=p5T+AA@mail.gmail.com>

--===============6504758777782277436==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64

RGFzIGlzdCBkZXIgZXJzdGUgVGVzdMOk

--===============6504758777782277436==--

17773cd99dd75d93
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 12:06:30 -0800
Content-Type: multipart/mixed; boundary="===============5236236051903402932=="
MIME-Version: 1.0
to: datathon@lumos-consulting.at
From: team4@lumos-consulting.at
subject: Test_0
Date: Fri, 5 Feb 2021 12:06:29 -0800
Message-Id: <C

In [14]:
message = create_message("datathon@lumos-consulting.at", "thomas.n71@gmail.com", "Test_0", "Das ist der erste Testä")

Content-Type: multipart/mixed; boundary="===============6349791082375090044=="
MIME-Version: 1.0
to: thomas.n71@gmail.com
from: datathon@lumos-consulting.at
subject: Test_0

--===============6349791082375090044==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64

RGFzIGlzdCBkZXIgZXJzdGUgVGVzdMOk

--===============6349791082375090044==--

b'Q29udGVudC1UeXBlOiBtdWx0aXBhcnQvbWl4ZWQ7IGJvdW5kYXJ5PSI9PT09PT09PT09PT09PT02MzQ5NzkxMDgyMzc1MDkwMDQ0PT0iCk1JTUUtVmVyc2lvbjogMS4wCnRvOiB0aG9tYXMubjcxQGdtYWlsLmNvbQpmcm9tOiBkYXRhdGhvbkBsdW1vcy1jb25zdWx0aW5nLmF0CnN1YmplY3Q6IFRlc3RfMAoKLS09PT09PT09PT09PT09PT02MzQ5NzkxMDgyMzc1MDkwMDQ0PT0KQ29udGVudC1UeXBlOiB0ZXh0L3BsYWluOyBjaGFyc2V0PSJ1dGYtOCIKTUlNRS1WZXJzaW9uOiAxLjAKQ29udGVudC1UcmFuc2Zlci1FbmNvZGluZzogYmFzZTY0CgpSR0Z6SUdsemRDQmtaWElnWlhKemRHVWdWR1Z6ZE1PawoKLS09PT09PT09PT09PT09PT02MzQ5NzkxMDgyMzc1MDkwMDQ0PT0tLQo='
Q29udGVudC1UeXBlOiBtdWx0aXBhcnQvbWl4ZWQ7IGJvdW5kYXJ5PSI9PT09PT09PT09PT09PT02MzQ5NzkxMDgyMzc1MDkwMDQ0

In [15]:
send_message(service, "me", message)

Message Id: 17773d2bcfa43966


In [16]:
messages = receive_all_messages(service) #receive_unread_messages(service)

17773d2bcfa43966
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 12:12:06 -0800
Content-Type: multipart/mixed; boundary="===============6349791082375090044=="
MIME-Version: 1.0
to: thomas.n71@gmail.com
From: team4@lumos-consulting.at
subject: Test_0
Date: Fri, 5 Feb 2021 12:12:06 -0800
Message-Id: <CAEzfA9qxUue951H6NGqiLvNGZr7f4kYjnVkyXzKWaRfysKC2_A@mail.gmail.com>

--===============6349791082375090044==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64

RGFzIGlzdCBkZXIgZXJzdGUgVGVzdMOk

--===============6349791082375090044==--

17773d095a0a02e1
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 12:09:45 -0800
Content-Type: multipart/mixed; boundary="===============6504758777782277436=="
MIME-Version: 1.0
to: datathon@lumos-consulting.at
From: team4@lumos-consulting.at
subject: Test_0
Date: Fri, 5 Feb 2021 12:09:45 -0800
Message-Id: <CAEzfA9rF

In [17]:
for message in messages: 
    print("New Message!")
    print_message(message)
    print()


New Message!
RGFzIGlzdCBkZXIgZXJzdGUgVGVzdMOk


New Message!
RGFzIGlzdCBkZXIgZXJzdGUgVGVzdMOk


New Message!
RGFzIGlzdCBkZXIgZXJzdGUgVGVzdMOk


New Message!
RGFzIGlzdCBkZXIgZXJzdGUgVGVzdMOk


New Message!
Das ist der erste Test

New Message!
Das ist der erste Test

New Message!
RGFzIGlzdCBkZXIgZXJzdGUgVGVzdA==


New Message!
RGFzIGlzdCBkZXIgZXJzdGUgVGVzdA==


New Message!
RGFzIGlzdCBkZXIgZXJzdGUgVGVzdA==


New Message!
RGFzIGlzdCBkZXIgZXJzdGUgVGVzdA==


New Message!
RGFzIGlzdCBkZXIgZXJzdGUgVGVzdA==


New Message!
RGFzIGlzdCBkZXIgZXJzdGUgVGVzdA==


New Message!
Das ist der erste Test

New Message!
SWNoIGhhYmUgV0FSRUhPVVNFIFByb2R1a3RlIGltIElUJk9GRklDRUthdGFsb2cxNSBmaXJzdGNs
YXNzIHN0YW5kYXJkbWFlc3NpZyBpbnN0YWxsaWVydC4gRGllIEluc3RhbGxhdGlvbiBsaWVmIGZl
aGxlcmZyZWkgYWIgdW5kIHd1cmRlIG9yZG51bmdzZ2VtYWVzcyBhYmdlc2NobG9zc2VuLiBEYXMg
UHJvZ3JhbW0gd3VyZGUgYW5zY2hsaWVzc2VuZCByZWdpc3RyaWVydC4gIEJlaW0gUHJvZ3JhbW1h
dWZydWYga29tbXQgZnVlciBjYS4gNC01IFNla3VuZGVuIGRpZSBTYW5kdWhyIGF1ZiBkZW0gQmls
ZHNjaGlybSw

In [18]:
traindf=pd.read_csv("data/train.csv",sep=",")
traindf.iloc[0]["Narrative"]

' Ich bin begeistert von Ihren Programmen und nutze Windows 7 auf einem Notebook (was aber etwas umständlich ist, da zu klein) und einen iMac, welcher aufgrund der Größe wesentlich komfortabler ist. Sind Ihre Programme Apple-kompatibel? ...oder gibt es da etwaige Probleme oder Einschränkungen? Über eine schnelle Antwort würde ich mich freuen, da ich darauf brenne Ihre Programme zu nutzen.      '

In [ ]:
import time
while True:
    
  receive_unread_messages(service)
  time.sleep(30)